Math 5750/6880: Mathematics of Data Science \
Project 3

# 1. Fashion-MNIST image classification using sklearn

In [1]:
from tensorflow.keras.datasets import fashion_mnist
from sklearn.preprocessing import StandardScaler

# Load Fashion-MNIST
# Classes (0-9): T-shirt/top, Trouser, Pullover, Dress, Coat, Sandal, Shirt, Sneaker, Bag, Ankle boot
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
X_train = X_train.reshape(len(X_train), -1)
X_test  = X_test.reshape(len(X_test), -1)

# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [2]:
import warnings
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.exceptions import ConvergenceWarning

warnings.filterwarnings('ignore', category=ConvergenceWarning)

In [11]:
def run_configuration(config):
    mlp = MLPClassifier(**config)
    mlp.fit(X_train, y_train)
    y_pred = mlp.predict(X_test)
    test_item = {"config":config, "accuracy":accuracy_score(y_test, y_pred)}
    return test_item

In [13]:
import random

config_ranges = {
    "hidden_layer_sizes": [(100,), (10, 10)],
    "max_iter": [3],
    "alpha": [1e-3, 1e-4, 1e-5],
    "solver": ["adam", "sgd"],
    "learning_rate_init": [0.05, 0.1, 0.15],
    "activation": ['relu', 'logistic', 'tanh'],
    "verbose": [0],
    "random_state": [1]
}

population = []

def sort_population():
    global population
    population = sorted(population, key=lambda item: -item["accuracy"])

def add_to_population(config):
    result = run_configuration(config)
    print(result)
    population.insert(1, result)

for _ in range(0, 12):
    new_config = {}
    for each_key in config_ranges:
        new_config[each_key] = random.choice(config_ranges[each_key])
    add_to_population(new_config)








{'config': {'hidden_layer_sizes': (10, 10), 'max_iter': 3, 'alpha': 1e-05, 'solver': 'adam', 'learning_rate_init': 0.15, 'activation': 'logistic', 'verbose': 0, 'random_state': 1}, 'accuracy': 0.7339}
{'config': {'hidden_layer_sizes': (10, 10), 'max_iter': 3, 'alpha': 0.001, 'solver': 'adam', 'learning_rate_init': 0.15, 'activation': 'logistic', 'verbose': 0, 'random_state': 1}, 'accuracy': 0.7389}
{'config': {'hidden_layer_sizes': (100,), 'max_iter': 3, 'alpha': 0.0001, 'solver': 'adam', 'learning_rate_init': 0.1, 'activation': 'logistic', 'verbose': 0, 'random_state': 1}, 'accuracy': 0.7893}
{'config': {'hidden_layer_sizes': (10, 10), 'max_iter': 3, 'alpha': 0.001, 'solver': 'adam', 'learning_rate_init': 0.05, 'activation': 'logistic', 'verbose': 0, 'random_state': 1}, 'accuracy': 0.7915}
{'config': {'hidden_layer_sizes': (10, 10), 'max_iter': 3, 'alpha': 1e-05, 'solver': 'adam', 'learning_rate_init': 0.15, 'activation': 'relu', 'verbose': 0, 'random_state': 1}, 'accuracy': 0.1959}
{

In [8]:



for solver in ['adam', 'sgd']:
    for activation in ['relu', 'logistic', 'tanh']:
        for hidden_layer_sizes in [(100), (10, 10)]:
            new_config = default_config.copy()
            new_config["solver"] = solver
            new_config["activation"] = activation
            new_config["hidden_layer_sizes"] = hidden_layer_sizes
            result = run_configuration(new_config)
            results.insert(1, result)
            print(result)


{'config': {'hidden_layer_sizes': 100, 'max_iter': 3, 'alpha': 0.0001, 'solver': 'adam', 'learning_rate_init': 0.1, 'activation': 'relu', 'verbose': 0, 'random_state': 1}, 'accuracy': 0.7908}
{'config': {'hidden_layer_sizes': (10, 10), 'max_iter': 3, 'alpha': 0.0001, 'solver': 'adam', 'learning_rate_init': 0.1, 'activation': 'relu', 'verbose': 0, 'random_state': 1}, 'accuracy': 0.4222}
{'config': {'hidden_layer_sizes': 100, 'max_iter': 3, 'alpha': 0.0001, 'solver': 'adam', 'learning_rate_init': 0.1, 'activation': 'logistic', 'verbose': 0, 'random_state': 1}, 'accuracy': 0.7893}
{'config': {'hidden_layer_sizes': (10, 10), 'max_iter': 3, 'alpha': 0.0001, 'solver': 'adam', 'learning_rate_init': 0.1, 'activation': 'logistic', 'verbose': 0, 'random_state': 1}, 'accuracy': 0.7479}
{'config': {'hidden_layer_sizes': 100, 'max_iter': 3, 'alpha': 0.0001, 'solver': 'adam', 'learning_rate_init': 0.1, 'activation': 'tanh', 'verbose': 0, 'random_state': 1}, 'accuracy': 0.7249}
{'config': {'hidden_la

In [9]:
sort_results()

for each_result in results[:len(results)//3]:
    for alpha in [1e-5,1e-3]:
        for learning_rate_init in [0.05, 0.15]:
            new_config = each_result["config"].copy()
            new_config["alpha"] = alpha
            new_config["learning_rate_init"] = learning_rate_init
            result = run_configuration(new_config)
            results.insert(1, result)
            print(result)



{'config': {'hidden_layer_sizes': 100, 'max_iter': 3, 'alpha': 1e-05, 'solver': 'sgd', 'learning_rate_init': 0.05, 'activation': 'logistic', 'verbose': 0, 'random_state': 1}, 'accuracy': 0.8638}
{'config': {'hidden_layer_sizes': 100, 'max_iter': 3, 'alpha': 1e-05, 'solver': 'sgd', 'learning_rate_init': 0.15, 'activation': 'logistic', 'verbose': 0, 'random_state': 1}, 'accuracy': 0.8726}
{'config': {'hidden_layer_sizes': 100, 'max_iter': 3, 'alpha': 0.001, 'solver': 'sgd', 'learning_rate_init': 0.05, 'activation': 'logistic', 'verbose': 0, 'random_state': 1}, 'accuracy': 0.8638}
{'config': {'hidden_layer_sizes': 100, 'max_iter': 3, 'alpha': 0.001, 'solver': 'sgd', 'learning_rate_init': 0.15, 'activation': 'logistic', 'verbose': 0, 'random_state': 1}, 'accuracy': 0.8725}
{'config': {'hidden_layer_sizes': 100, 'max_iter': 3, 'alpha': 1e-05, 'solver': 'sgd', 'learning_rate_init': 0.05, 'activation': 'relu', 'verbose': 0, 'random_state': 1}, 'accuracy': 0.874}
{'config': {'hidden_layer_size

In [10]:
sort_results()
for result in results:
    print(result)

{'config': {'hidden_layer_sizes': 100, 'max_iter': 3, 'alpha': 1e-05, 'solver': 'sgd', 'learning_rate_init': 0.05, 'activation': 'relu', 'verbose': 0, 'random_state': 1}, 'accuracy': 0.874}
{'config': {'hidden_layer_sizes': 100, 'max_iter': 3, 'alpha': 0.001, 'solver': 'sgd', 'learning_rate_init': 0.05, 'activation': 'relu', 'verbose': 0, 'random_state': 1}, 'accuracy': 0.8726}
{'config': {'hidden_layer_sizes': 100, 'max_iter': 3, 'alpha': 1e-05, 'solver': 'sgd', 'learning_rate_init': 0.15, 'activation': 'logistic', 'verbose': 0, 'random_state': 1}, 'accuracy': 0.8726}
{'config': {'hidden_layer_sizes': 100, 'max_iter': 3, 'alpha': 0.001, 'solver': 'sgd', 'learning_rate_init': 0.15, 'activation': 'logistic', 'verbose': 0, 'random_state': 1}, 'accuracy': 0.8725}
{'config': {'hidden_layer_sizes': 100, 'max_iter': 3, 'alpha': 1e-05, 'solver': 'sgd', 'learning_rate_init': 0.05, 'activation': 'tanh', 'verbose': 0, 'random_state': 1}, 'accuracy': 0.8705}
{'config': {'hidden_layer_sizes': 100,

# 3. Fashion-MNIST image classification  using pytorch

In [ ]:
import numpy as np
from tensorflow.keras.datasets import fashion_mnist
import torch
from torch.utils.data import TensorDataset, DataLoader

# Load Fashion-MNIST
# Classes (0-9): T-shirt/top, Trouser, Pullover, Dress, Coat, Sandal, Shirt, Sneaker, Bag, Ankle boot
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

# scale to [0,1], add channel dimension -> (N, 1, 28, 28)
X_train = (X_train.astype("float32") / 255.0)[:, None, :, :]
X_test  = (X_test.astype("float32")  / 255.0)[:,  None, :, :]

y_train = y_train.astype(np.int64)
y_test  = y_test.astype(np.int64)

# train/val split: last 10k of train as validation
X_tr, X_val = X_train[:50000], X_train[50000:]
y_tr, y_val = y_train[:50000], y_train[50000:]

# wrap in PyTorch TensorDatasets and DataLoaders
train_ds = TensorDataset(torch.from_numpy(X_tr),  torch.from_numpy(y_tr))
val_ds   = TensorDataset(torch.from_numpy(X_val), torch.from_numpy(y_val))
test_ds  = TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test))

train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
val_loader   = DataLoader(val_ds,   batch_size=256, shuffle=False)
test_loader  = DataLoader(test_ds,  batch_size=256, shuffle=False)

In [ ]:
import torch.nn as nn
import torch.optim as optim

# In colab, you should ``change runtime type'' to GPU.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# your code here